In [1]:
import numpy as np

# prepare training data
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
y = np.dot(X, np.array([1, 2])) + 3

# prepare evaluation data
X_eval = np.array([[3, 3], [3, 4]])
y_eval = np.dot(X_eval, np.array([1, 2])) + 3

In [2]:
from example import spark

22/11/11 23:05:43 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.31.194 instead (on interface wlx1cbfce3ffbfe)
22/11/11 23:05:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/sadhen/.ivy2/cache
The jars for the packages stored in: /home/sadhen/.ivy2/jars
ai.eto#rikai_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e7be51c5-c447-4bbb-ab18-2eecfa0cdab9;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/sadhen/.cache/pants/named_caches/pex_root/installed_wheels/233ee78b2fff1a37a858ec76db084cea1117898399f4665140a79bb72f0e709a/pyspark-3.2.2-py2.py3-none-any.whl/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found ai.eto#rikai_2.12;0.1.14 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found com.typesafe.scala-logging#scala-logging_2.12;3.9.4 in central
	found org.slf4j#slf4j-api;1.7.30 in local-m2-cache
	found org.mlflow#mlflow-client;1.21.0 in central
	found org.apache.logging.log4j#log4j-core;2.17.1 in spark-list
:: resolution report :: resolve 157ms :: artifacts dl 7ms
	:: modules in use:
	ai.eto#rikai_2.12;0.1.14 from central in [default]
	com.typesafe.scala-logging#scala-logging_2.12;3.9.4 from central in [default]
	org.apache.logging.log4j#log4j-core;2.17.1 from spark-list in [default]
	org.mlflow#mlflow-client;1.21.0 from central in [default]
	org.slf4j#slf4j-api;1.7.30 from local-m2-cache in [default]
	org.xerial.snappy#snappy-java;1.1.8.4 from central in [default]
	:: evicted modules:
	org.slf4j#slf4j-api;1.7.25 by [org.slf4j#slf4j-api;1.7.30] in [default]
	---------------------------------------------------------------------
	|                  |            

In [3]:
import getpass
import mlflow
import rikai
from sklearn.linear_model import LogisticRegression

mlflow_tracking_uri = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(mlflow_tracking_uri)

# enable autologging
mlflow.sklearn.autolog()

# train a model
model = LogisticRegression()
with mlflow.start_run() as run:
    ####
    # Part 1: Train the model and register it on MLflow
    ####
    model.fit(X, y)
    metrics = mlflow.sklearn.eval_and_log_metrics(model, X_eval, y_eval, prefix="val_")

    registered_model_name = f"{getpass.getuser()}_sklearn_log_r"
    rikai.mlflow.sklearn.log_model(
        model,
        "model",
        registered_model_name=registered_model_name,
        model_type="rikai_sklearn.LogisticsRegression",
    )


2022/11/11 23:05:45 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/11/11 23:05:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/sadhen/.cache/pants/named_caches/pex_root/venvs/009a4fd09a3040021b925a721520220a94401db6/1b6212c3def35029d3805c41e131d1f544e72526/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2022/11/11 23:05:48 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
/home/sadhen/.cache/pants/named_caches/pex_root/venvs/009a4fd09a3040021b925a721520220a94401db6/1b6212c3def35029d3805c41e131d1f544e72526/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precisio

## Create the model using the registered MLflow uri

In [4]:
from rikai.spark.sql.codegen.mlflow_logger import CONF_MLFLOW_TRACKING_URI

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set(CONF_MLFLOW_TRACKING_URI, mlflow_tracking_uri)
spark.sql(f"""
CREATE OR REPLACE MODEL mlflow_sklearn_m USING 'mlflow:///{registered_model_name}';
"""
)

spark.sql("show models").show(1, vertical=False, truncate=False)

+----------------+------+------------------------------+-------+
|name            |flavor|uri                           |options|
+----------------+------+------------------------------+-------+
|mlflow_sklearn_m|      |mlflow:///sadhen_sklearn_log_r|       |
+----------------+------+------------------------------+-------+



## Predict using the registered Rikai model

In [5]:
result = spark.sql(f"""
    select ML_PREDICT(mlflow_sklearn_m, array(1, 2))
""")

result.printSchema()
result.toPandas()

root
 |-- mlflow_sklearn_m: integer (nullable = true)



,mlflow_sklearn_m
0,8
